## Hangman Implementation

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. 

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('whatever', human, 8, True)

## Pre-processing
Use NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

1. compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. 

2. **Lowercase the words**. 

3. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. Both `training_set` and `test_set` should be a python `list`. Besides, `test_set` should contain 1000 word types.

In [5]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []

#words from brown corpus
brown_words = brown.words()

#lowercase the corpus and remove the word which contain non-alphabetic characters
processed_words = []
for word in brown_words:
    if word.isalpha():
        processed_words.append(word.lower())

#unique words in brown corpus
unique_words = list(set(processed_words))

# change to array in order to conduct np shuffle, then convert to list
unique_words = np.array(unique_words)
np.random.shuffle(unique_words)
unique_words = unique_words.tolist()

test_set = unique_words[:1000]
training_set = unique_words[1000:]

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


**Play the game by yourself. Make sure that `interactive` = True**

In [6]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Baseline Method

Baseline is a trivial **random method**. The method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

`test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

In [7]:
def test_guesser(guesser, test=test_set):
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [8]:
import string
import random

def random_guesser(mask, guessed, **kwargs):
    
    # available is a list that does not contain the character in guessed
    available = set(string.ascii_lowercase) - guessed
    return random.choice(list(available))

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = newt
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.728


## Unigram Language Model

The first real AI is a **unigram language model** over the training set. This requires to find the frequencies of characters over all training words. A guesser should returns the character with the highest probability. Remember to exclude already guessed characters. 

In [9]:
unigram_counts = None

from collections import Counter

# Build unigram model. Count character frequency.
def unigram(corpus):
    unigram_counts = Counter()
    
    for word in corpus:
        for char in word:
            unigram_counts[char] += 1
            
    return unigram_counts

unigram_counts = unigram(training_set)

In [10]:
def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    
    copy_dict = unigram_counts.copy() 
    
    # Delete the guessed word first, then take the max value in the dictionary
    for char in guessed:
        del copy_dict[char]

    return max(copy_dict, key=copy_dict.get)

result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.484


## Unigram Based on Word Length

The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. 

When encounter an unseen word length, the method should behave like the previous `unigram_guesser`.

In [11]:
unigram_counts_by_length = None

from collections import defaultdict, Counter
def unigram_length(corpus):
    #This allow all values are counters
    unigram_counts = defaultdict(Counter)
    
    for word in corpus:
        length = len(word)
        for char in word:
            #index will be[word's length][character]
            unigram_counts[length][char] += 1
            
    return unigram_counts

unigram_counts_by_length = unigram_length(training_set)

###############################################################################################################################
# Add the unseen character for each length to zero in order to avoid max() has no element to pick, since this algo will delete#
# element in dictionary and take max value of it. If we encounter unseen pattern, max() will have error with no arguement.    #
###############################################################################################################################

for key in unigram_counts_by_length.keys():
    if not len(unigram_counts_by_length[key]) == 26:
        add_char = set(string.ascii_lowercase) - set(list(unigram_counts_by_length[key].keys()))
    
        for char in add_char:
            unigram_counts_by_length[key][char] = 0

In [12]:
def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):

    if len(mask) not in list(unigram_counts_by_length.keys()):
        return unigram_guesser(mask, guessed, unigram_counts)
    else:
        copy_dict = unigram_counts_by_length[len(mask)].copy() 
        
        #remove the guessed characters in copy_dict and then return the max frequency character.
        for char in guessed:
            del copy_dict[char]
        
        return max(copy_dict, key=copy_dict.get)

    
result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.443


## Bigram Language Model

Now build a **bigram language model** over characters, and train it over the training words. The order of characters is important. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly, the distribution over characters that start or end a word are highly biased.

**Bigram guesser should apply language model to each blank position in the secret word by using its left context character.**

For example, `e _ c _ b _ _` , 

The left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, revert to using a unigram language model. We need sum up the probability distribution (over all alphabets from a to z) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

In [13]:
bigram_counts = None

# add $ to the front of a word since this is a Bigram Model.
def convert_word(word):
    return "$" + word

# Collect bigram counts. Because we don't delete keys in dictionary and counter will return 0 for unseen pattern,
# we don't need to add char for dictionary as Unigram Based on Word Length approach.
def bigram(corpus):
    bigram_counts = defaultdict(Counter)
    
    for word in corpus:
        word = convert_word(word)
        
        # generate a list of bigrams
        bigram_list = zip(word[:-1], word[1:])
        
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
    return bigram_counts
    
bigram_counts = bigram(training_set)

In [14]:
# Calculate bigram probability
def bigram_prob(key, char, bigram_counts):
    prev_word_counts = bigram_counts[key]
    total_counts = float(sum(prev_word_counts.values()))

    return prev_word_counts[char] / float(sum(prev_word_counts.values()))

In [15]:
def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts=unigram_counts): # add extra arguments if needed
    
    # available is a list that does not contain the character in guessed
    available = list(set(string.ascii_lowercase) - guessed)
    
    # The probabilities of available character
    bigram_probs = []
    
    for char in available:
        char_prob = 0
        for index in range(len(mask)):
            # The first case is that the first char has not been guessed
            if index == 0 and mask[index] == '_':
                char_prob +=  bigram_prob('$', char, bigram_counts)
                
            # The second case is that the other chars have not been guessed
            elif mask[index] == '_':
                # if the previous word has been guessed apply bigram
                if not mask[index - 1] == '_':
                    char_prob +=  bigram_prob(mask[index - 1], char, bigram_counts)
                    
                # If the previous word has not been guessed apply unigram
                else:
                    char_prob +=  unigram_counts[char] / float(sum(unigram_counts.values()))
                
            # The final case is that the character is guessed so we skip this position
            else:
                continue
                
        bigram_probs.append(char_prob)
    
    # Return the max probability of char
    return available[bigram_probs.index(max(bigram_probs))]

result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.641


## Trigram Model with Smoothing Method

1. Pad \$\$ to the front of the words
2. Get the unigram, bigram and trigram count from training set
3. Apply trigram model to each blank position in the secret word by using its left 2 context characters
4. We need to apply trigram, bigram and unigram based on different situations.
    - For the first character
        - \$\$_ : apply trigram for the blank
    - For the second character
        - \$a\_ : apply trigram for the blank
        - \$\_ \_: apply unigram for the last blank, since we cannot apply bigram and trigram.
    - For the other position words
        - aa_: apply trigram for the blank
        - \_a\_: apply bigram for the last blank, since we cannot apply trigram
        - a\_ \_: apply unigram for the last blank, since we cannot apply bigram and trigram
5. Apply interpolation smoothing method. 
    - For trigram probability, multiply 0.6
    - For bigram probability, multiply 0.3
    - For unigram probability, multiply 0.1
6. Sum up the probability distribution (over all alphabets from a to z) for every blank.
7. Take the max probability of the character

In [16]:
# add $$ to the front of a word
def trigram_convert_word(word):
    return "$$" + word

# collect trigram counts
def trigram(corpus):
    trigram_counts = Counter()
    bigram_counts = defaultdict(Counter)
    
    for word in corpus:
        word = trigram_convert_word(word)
        
        # generate a list of trigrams
        trigram_list = zip(word[:-2], word[1:-1], word[2:])
        
        # generate a list of bigrams
        bigram_list = zip(word[:-1], word[1:])
        
        # iterate over trigrams
        for trigram in trigram_list:
            first, second, third = trigram
            element = first+second+third
            trigram_counts[element] += 1
            
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
            
    return trigram_counts, bigram_counts
    
trigram_counts, bigram_counts_for_trigram = trigram(training_set)

In [17]:
# Calculate trigram probability
def trigram_prob(wi_2, wi_1, char, trigram_counts, bigram_counts):
    return trigram_counts[wi_2 + wi_1 + char] / float(bigram_counts[wi_2][wi_1])

In [18]:
def my_amazing_ai_guesser(mask, guessed, bigram_counts=bigram_counts_for_trigram, trigram_counts=trigram_counts, 
                          unigram_counts=unigram_counts):
    
    # available is a list that does not contain the character in guessed
    available = list(set(string.ascii_lowercase) - guessed)

    # The probabilities of available character
    trigram_probs = []
    
    # if len(mask) = 1, means that there is only a character. Therefore, need to pad in order to avoid error from 
    # traverse mask[index -2] and mask[index -1] 
    mask = ['$', '$'] + mask
    
    trigram_lambda = 0.6
    bigram_lambda = 0.3
    unigram_lambda = 0.1
    
    for char in available:
        char_prob = 0
        for index in range(len(mask)):
            # The first case is that the first char has not been guessed
            if index == 0 and mask[index] == '_':
                char_prob += trigram_lambda * trigram_prob('$', '$', char, trigram_counts, bigram_counts)
                
            # The second case is that the second char has not been guessed
            if index == 1 and mask[index] == '_':
                # If the previous word has been guessed, apply trigram
                if not mask[index - 1] == '_':
                    char_prob += trigram_lambda * trigram_prob('$', mask[index - 1], char, trigram_counts, bigram_counts)
                    
                # If the previous word has not been guessed, apply unigram
                else:
                    char_prob +=  unigram_lambda * unigram_counts[char] / float(sum(unigram_counts.values()))
            
            # The third case is that the other chars have not been guessed
            elif mask[index] == '_':
                # If wi-2 and wi-1 have been guessed, apply trigram
                if not mask[index - 2] == '_' and not mask[index - 1] == '_':
                    char_prob += trigram_lambda * trigram_prob(mask[index - 2], mask[index - 1], char, 
                                                            trigram_counts, bigram_counts)
                    
                # If wi-2 hasn't been guessed but wi-1 has been guessed, apply bigram
                elif mask[index - 2] == '_' and not mask[index - 1] == '_':
                    char_prob += bigram_lambda * bigram_prob(mask[index - 1], char, bigram_counts)
                
                # If wi-1 hasn't been guessed, apply unigram
                else:
                    char_prob +=  unigram_lambda * unigram_counts[char] / float(sum(unigram_counts.values()))
                
            # The final case is that the character is guessed so we skip this position
            else:
                continue
                
        trigram_probs.append(char_prob)
    
    # Return the max probability of char
    return available[trigram_probs.index(max(trigram_probs))]

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  8.102
